## CS215 Assignment 2
### 210010076 Shantanu Welling
### 210050044 Dhananjay Raman

Importing all relevant libraries and defining constants

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import h5py

In [12]:
#Set random seed
np.random.seed(10)

#### Q1 (b)

In [13]:
theta=np.random.uniform(size=10000000) #Sample 10^7 points between 0 and 1
theta*=2*np.pi #Map them to 0 to 2*pi, they represent the polar angle

In [14]:
r=np.random.uniform(size=10000000) 
#Sample 10^7 points between 0 & 1, representing 
#r in polar co-ordinates
r=np.sqrt(r) #Use CDF inverse method for sqrt(r) as mentioned in the algorithm
x=[]
y=[]
for i in range(len(r)):
    x.append(r[i]*np.cos(theta[i])) #append x co-ordinate, already set to scale
    y.append(r[i]*np.sin(theta[i])*0.5) 
    #append y co-ordinate and scale it to minor axis length

In [15]:
plt.hist2d(x,y,bins=500,density=True,cmap=plt.cm.Reds)
plt.colorbar()
plt.gca().set_aspect('equal', adjustable='box')
plt.title("Ellipse Sample Points Histogram")
plt.xlabel("X axis")
plt.ylabel("Y axis")
plt.savefig("../results/ellipse_hist.jpg", dpi=600)
plt.close();

#### Q1 (c)

In [16]:
u=np.random.uniform(size=10000000) #Generate 10^7 uniform random numbers betwen 0 & 1
v=np.random.uniform(size=10000000) #To scale the 2 side length vectors
resx=[] #Result x co-ordinates
resy=[] #Result y co-ordinates

In [17]:
side1=((np.pi/3)**2+(np.e)**2)**(0.5) #Length of side between (0,0) and (pi/3,e)
side2=np.pi #Length of side along x axis
ang1=np.arctan(np.e*3/np.pi) #Angle between the 2 sides

In [18]:
for i in range(len(u)):
    if(u[i]+v[i]<=1): #If sum of 2 scaled side length vectors lies in the triangle
        resx.append(v[i]*side2+u[i]*side1*np.cos(ang1)) 
        resy.append(u[i]*side1*np.sin(ang1))
        #Append the x & y co-ordinates of the 2 summed vectors
    else: 
        #If sum of 2 scaled side length vectors lies in the other half of the
        #parallelogram spanned by the 2 side vectors, reflect it along the 
        #parallelogram diagonal
        resx.append((1-v[i])*side2+ (1-u[i])*side1*np.cos(ang1))
        resy.append((1-u[i])*side1*np.sin(ang1))

In [19]:
plt.hist2d(resx,resy,bins=500,density=True, cmap=plt.cm.Reds)
plt.colorbar()
plt.title("Triangle Sample Points Histogram")
plt.xlabel("X axis")
plt.ylabel("Y axis")
plt.savefig("../results/triangle_hist.jpg", dpi=600)
plt.close();

#### Q2

In [20]:
C=np.array([[1.6250,-1.9486],[-1.9486,3.8750]]) #Covariance matrix
mu=np.array([1,2]) #Mean matrix
nlist=[10,100,1000,10000,100000] #List of Ns

In [21]:
evalues,evecs=np.linalg.eig(C) #Compute eigenvalues of Covariance matric
diagmat = np.array([[evalues[0],0],[0,evalues[1]]]) #Compute Diagonal Matrix

In [22]:
A = np.matmul(evecs,diagmat**0.5); #Using C=AA^T, compute A 
# C = QDQ^T (Since C is real symmetric)
# Where Q (orthogonal) is matrix of eigenvectors of C
# D is diagonal matrix of eigen values of C
# C = (QD^(1/2))(QD^(1/2))^T = AA^T
# A = QD^(1/2)

In [23]:
xlist=[] #List of all points for all Ns
for n in nlist:
    Xnlist=np.zeros(shape=(2,n)) #List of points generated for a given N
    for i in range(n):
        w1=np.random.normal()
        w2=np.random.normal()
        W=np.array([w1,w2])
        X=np.matmul(A,np.transpose(W))+np.transpose(mu) #X=AW+u
        Xnlist[:,i]=X
    xlist.append(Xnlist) #Append N points list to final list

In [24]:
xmeans=[] #MLE Mean for all Ns
xcovs=[] #MLE Covariance for all Ns
for i in range(len(xlist)):
    xmeans.append(np.sum(xlist[i], axis=1)/len(xlist[i][0])) #Sum all the points for a given N and divide by N
    covar=np.zeros(shape=(len(xlist[i]),len(xlist[i])))
    l=len(xlist[i][0])
    for j in range(len(xlist[i])):
        for k in range(len(xlist[i])): #Compute Covariance
            covar[j,k]=np.sum(xlist[i][j,:]*xlist[i][k,:])/l-np.sum(xlist[i][j,:])*np.sum(xlist[i][k,:])/l**2
    xcovs.append(covar) #Append computed covariance matrix to the list


### Q2 (b) & (c)

In [25]:
means=np.zeros(shape=(5,100)) #List for errors in mean
covarerr=np.zeros(shape=(5,100)) #List for errors in Covariance matrix
Cnorm=np.linalg.norm(C, 'fro') #Norm of Covariance matrix
for t in range(100): #For each N repeeating experiment 100 times
    xlist=[]
    for n in nlist: #compute mean as above
        Xnlist=np.zeros(shape=(2,n))
        for i in range(n):
            w1=np.random.normal()
            w2=np.random.normal()
            W=np.array([w1,w2])
            X=np.matmul(A,np.transpose(W))+np.transpose(mu)
            Xnlist[:,i]=X
        xlist.append(Xnlist)
    for i in range(len(xlist)):
        m=np.sum(xlist[i], axis=1)/len(xlist[i][0])
        meanerror=np.linalg.norm(m-np.transpose(mu))/np.linalg.norm(np.transpose(mu)) #find error in mean
        means[i][t]=meanerror #append error in mean to the mean error list
        covar=np.zeros(shape=(len(xlist[i]),len(xlist[i]))) 
        l=len(xlist[i][0])
        for j in range(len(xlist[i])): #compute covariance matrix as above
            for k in range(len(xlist[i])):
                covar[j,k]=np.sum(xlist[i][j,:]*xlist[i][k,:])/l-np.sum(xlist[i][j,:])*np.sum(xlist[i][k,:])/l**2
        covarerr[i][t]=np.linalg.norm(covar-C, 'fro')/Cnorm #append error in covariance matrix to the corresponding list
    

In [26]:
#Boxplot for errors in Mean
data=[] 
for i in range(len(means)):
    data.append(means[i,:])
plt.boxplot(data);
plt.xlabel("$log_{10}N$")
plt.xticks(ticks=[1,2,3,4,5], labels=[1,2,3,4,5])
plt.title('Boxplot (Bivariate Gaussian Distribution)')
plt.ylabel('Standardized Error in Mean')
plt.savefig('../results/boxplot_bivariate_gauss_mean.jpg',dpi=600)
plt.close();

In [27]:
#Boxplot for errors in Covariance matrix
data2=[]
for i in range(len(covarerr)):
    data2.append(covarerr[i,:])
plt.boxplot(data2);
plt.xlabel("$log_{10}N$")
plt.xticks(ticks=[1,2,3,4,5], labels=[1,2,3,4,5])
plt.title('Boxplot (Bivariate Gaussian Distribution)')
plt.ylabel('Standardized Error in Covariance')
plt.savefig('../results/boxplot_bivariate_gauss_covar.jpg',dpi=600)
plt.close();

### Q2 (d)

In [28]:
xlist=[] #Computation logic same as Q2 (b) & (c)
for n in nlist:
    Xnlist=np.zeros(shape=(2,n))
    for i in range(n):
        w1=np.random.normal()
        w2=np.random.normal()
        W=np.array([w1,w2])
        X=np.matmul(A,np.transpose(W))+np.transpose(mu)
        Xnlist[:,i]=X
    xlist.append(Xnlist)
xmeans=[]
xcovs=[]
for i in range(len(xlist)):
    xmeans.append(np.sum(xlist[i], axis=1)/len(xlist[i][0]))
    covar=np.zeros(shape=(len(xlist[i]),len(xlist[i])))
    l=len(xlist[i][0])
    for j in range(len(xlist[i])):
        for k in range(len(xlist[i])):
            covar[j,k]=np.sum(xlist[i][j,:]*xlist[i][k,:])/l-np.sum(xlist[i][j,:])*np.sum(xlist[i][k,:])/l**2
    xcovs.append(covar)

In [29]:
fig, ax = plt.subplots(3,2, figsize=(10,20)) #subplots
fig.suptitle('Scatter Plots',size=20) #Scatter plots for each N of a single data sample
for i in range(len(xlist)):
    covar = xcovs[i]
    m = xmeans[i]
    evalues,evecs=np.linalg.eig(covar) #Computing eigenvalues and eigenvectors
    ax[i//2][i%2].scatter(xlist[i][0,:],xlist[i][1,:])
    #Plotting principal modes of variation
    ax[i//2][i%2].plot((m[0],m[0]+evecs[0][0]*np.sqrt(evalues[0])),(m[1],m[1]+evecs[1][0]*np.sqrt(evalues[0])),'k-',lw=3)
    ax[i//2][i%2].plot((m[0],m[0]+evecs[0][1]*np.sqrt(evalues[1])),(m[1],m[1]+evecs[1][1]*np.sqrt(evalues[1])),'r-',lw=3)
    ax[i//2][i%2].set_title(f"N = {nlist[i]}")
    ax[i//2][i%2].set_xlabel('X',size=12)
    ax[i//2][i%2].set_ylabel('Y',size=12)
    ax[i//2][i%2].set_aspect('equal', adjustable='box')
fig.delaxes(ax[2][1])
fig.savefig("../results/scatter_bivariate.jpg",dpi=800);
plt.close();

### Q3

In [30]:
arrays={}
mat = h5py.File('../data/points2D_Set1.mat')
for k, v in mat.items():
    arrays[k] = np.array(v)
points = np.array([np.array(arrays['x']),np.array(arrays['y'])])
points = points[:,0,:]
covar = np.zeros(shape=(2,2)) #Compute covariance matrix for sample points
for i in range(2):
    for j in range(2):
        l=len(points[0])
        covar[i][j]=np.sum(points[i,:]*points[j,:])/l - np.sum(points[i,:])*np.sum(points[j,:])/l**2
evalues,evecs=np.linalg.eig(covar) #Compute eigenvectors and eigenvalues
#Principal mode of variation is along the eigenvector having maximum eigenvalue
#Linear relationship between X and Y can be approximated by the direction of their principal mode
# of variation
if evalues[0]>evalues[1]:
    dirn = evecs[:,0]
else:
    dirn = evecs[:,1]
mu = np.sum(points,axis=1)/len(points[0]) #Mean of the sample set

In [31]:
plt.scatter(arrays['x'], arrays['y']) #Scatter plot of the dataset
#Line approximating linear relationship between the points
plt.plot((np.min(points[0,:]),np.max(points[0,:])),(mu[1]+(np.min(points[0,:])-mu[0])*dirn[1]/dirn[0],mu[1]+(np.max(points[0,:])-mu[0])*dirn[1]/dirn[0]),'r-',lw=2)
plt.xlabel("X axis")
plt.ylabel("Y axis")
plt.title("Scatter plot Set 1")
plt.savefig("../results/scatterplot_set1.jpg",dpi=600)
plt.close();

Second part same as above

In [32]:
arrays={}
mat = h5py.File('../data/points2D_Set2.mat')
for k, v in mat.items():
    arrays[k] = np.array(v)
points = np.array([np.array(arrays['x']),np.array(arrays['y'])])
points = points[:,0,:]
covar = np.zeros(shape=(2,2))
for i in range(2):
    for j in range(2):
        l=len(points[0])
        covar[i][j]=np.sum(points[i,:]*points[j,:])/l - np.sum(points[i,:])*np.sum(points[j,:])/l**2
evalues,evecs=np.linalg.eig(covar)
if evalues[0]>evalues[1]:
    dirn = evecs[:,0]
else:
    dirn = evecs[:,1]
mu = np.sum(points,axis=1)/len(points[0])

In [33]:
plt.scatter(arrays['x'], arrays['y'])
plt.plot((np.min(points[0,:]),np.max(points[0,:])),(mu[1]+(np.min(points[0,:])-mu[0])*dirn[1]/dirn[0],mu[1]+(np.max(points[0,:])-mu[0])*dirn[1]/dirn[0]),'r-',lw=2)
plt.xlabel("X axis")
plt.ylabel("Y axis")
plt.title("Scatter plot Set 2")
plt.savefig("../results/scatterplot_set2.jpg",dpi=600)
plt.close();

### Q4

In [34]:
arrays={}
mat = h5py.File('../data/mnist.mat') #Load and process data
for k, v in mat.items():
    arrays[k] = np.array(v, dtype=np.float64)
for i in range(len(arrays['digits_train'])):
    arrays['digits_train'][i]=np.transpose(arrays['digits_train'][i])
for i in range(len(arrays['digits_test'])):
    arrays['digits_test'][i]=np.transpose(arrays['digits_test'][i])

#### Q4 (i): Mean

In [35]:
digits=[i for i in range(10)] 
meanmat={key: np.zeros((28**2,1)) for key in digits} #Dictionary mapping each digit to its mean vector
sizemat={key: 0 for key in digits} #Dictionary mapping each digit to number of data samples of that digit

for i in range(len(arrays['digits_train'])):
    meanmat[arrays['labels_train'][0][i]]+=np.reshape(arrays['digits_train'][i],(28**2,1)) #Compute mean
    sizemat[arrays['labels_train'][0][i]]+=1 #Increment size
for key1 in sizemat.keys():
    meanmat[key1]/=sizemat[key1] 

In [36]:
fig, ax = plt.subplots(3,4, figsize=(10,8)) #subplots
fig.suptitle("Mean Matrices of Digits") #Plot mean of digits
for key1 in meanmat.keys():
    ax[key1//4][key1%4].imshow(np.reshape(meanmat[key1],(28,28)))
fig.delaxes(ax[2][2])
fig.delaxes(ax[2][3])
plt.savefig("../results/digitsmean.jpg",dpi=600)
plt.close();


#### Q4 (ii) Covariance

In [37]:
covmat={key: np.zeros((28**2,28**2)) for key in digits} #Dictionary mapping digits to their covariance matrix
for i in range(len(arrays['digits_train'])):
    covmat[arrays['labels_train'][0][i]]+=np.matmul(np.reshape(arrays['digits_train'][i],(28**2,1)),np.transpose(np.reshape(arrays['digits_train'][i],(28**2,1))))
for key1 in covmat.keys():
    covmat[key1]/=(sizemat[key1]-1) #(n-1) as sample covariance
    covmat[key1]-=np.matmul(meanmat[key1],np.transpose(meanmat[key1])) #Compute covariance

In [38]:
eiglist={key: 0 for key in digits} #Dictionary mapping digits to their eigenvalues
for key1 in digits:
    eiglist[key1]=np.sort(np.linalg.eigvalsh(covmat[key1]))

Principal mode of variation of each digit

In [39]:
eigval_principal={key: 0 for key in digits} #Principal eigenvalue for each digit
eigvecs_principal={key: None for key in digits} #Principal eigenvector for each digit
for k in covmat.keys():
    eigv,eigvecs=np.linalg.eigh(covmat[k])
    eigval_principal[k]= np.max(eigv)
    indmax=np.argmax(eigv)
    eigvecs_principal[k]=np.reshape(eigvecs[:,indmax],(28**2,1))

Graph of eigenvalues for each digit

In [40]:
xs=[i for i in range(1,28**2+1)]
fig,ax=plt.subplots(4,3,figsize=(23,23))
for key1 in eiglist.keys():
    ax[key1//3][key1%3].plot(xs,np.flip(eiglist[key1]), 'b.')
    ax[key1//3][key1%3].set_ylabel("Eigenvalues")
    ax[key1//3][key1%3].set_xlabel("Index of eigenvalue")
    ax[key1//3][key1%3].set_title(f"Digit {key1}")
fig.delaxes(ax[3][1])
fig.delaxes(ax[3][2])
plt.savefig("../results/digits_eigvals.jpg",dpi=400)
plt.close();

Principal mode of variation of digits around their mean and their plots

In [41]:
fig,ax=plt.subplots(3,3,figsize=(12,12))
for key1 in range(3):
    eigv,eigvect=np.linalg.eigh(covmat[key1])
    maxind=np.argmax(eigv)
    maxeigvec=eigvect[:,maxind]
    maxeigvec=np.reshape(maxeigvec,(28**2,1))
    res1=np.reshape((meanmat[key1]-np.sqrt(max(eigv))*maxeigvec), (28,28))
    res2=np.reshape((meanmat[key1]+np.sqrt(max(eigv))*maxeigvec), (28,28))
    res3=np.reshape(meanmat[key1], (28,28))
    ax[key1][0].imshow(res1)
    ax[key1][0].set_title("$\u03BC - v_{1}\u221A\u03BB_{1}$")
    ax[key1][1].imshow(res3)
    ax[key1][1].set_title("\u03BC")
    ax[key1][2].imshow(res2)
    ax[key1][2].set_title("$\u03BC + v_{1}\u221A\u03BB_{1}$")
plt.savefig("../results/pmv_digit0_2.jpg",dpi=600)
plt.close();

In [42]:
fig,ax=plt.subplots(3,3,figsize=(12,12))
for key1 in range(3,6):
    eigv,eigvect=np.linalg.eigh(covmat[key1])
    maxind=np.argmax(eigv)
    maxeigvec=eigvect[:,maxind]
    maxeigvec=np.reshape(maxeigvec,(28**2,1))
    res1=np.reshape((meanmat[key1]-np.sqrt(max(eigv))*maxeigvec), (28,28))
    res2=np.reshape((meanmat[key1]+np.sqrt(max(eigv))*maxeigvec), (28,28))
    res3=np.reshape(meanmat[key1], (28,28))
    ax[key1%3][0].imshow(res1)
    ax[key1%3][0].set_title("$\u03BC - v_{1}\u221A\u03BB_{1}$")
    ax[key1%3][1].imshow(res3)
    ax[key1%3][1].set_title("\u03BC")
    ax[key1%3][2].imshow(res2)
    ax[key1%3][2].set_title("$\u03BC + v_{1}\u221A\u03BB_{1}$")
plt.savefig("../results/pmv_digit3_5.jpg",dpi=600)
plt.close();

In [43]:
fig,ax=plt.subplots(4,3,figsize=(12,15))
for key1 in range(6,10):
    eigv,eigvect=np.linalg.eigh(covmat[key1])
    maxind=np.argmax(eigv)
    maxeigvec=eigvect[:,maxind]
    maxeigvec=np.reshape(maxeigvec,(28**2,1))
    res1=np.reshape((meanmat[key1]-np.sqrt(max(eigv))*maxeigvec), (28,28))
    res2=np.reshape((meanmat[key1]+np.sqrt(max(eigv))*maxeigvec), (28,28))
    res3=np.reshape(meanmat[key1], (28,28))
    ax[key1%6][0].imshow(res1)
    ax[key1%6][0].set_title("$\u03BC - v_{1}\u221A\u03BB_{1}$")
    ax[key1%6][1].imshow(res3)
    ax[key1%6][1].set_title("\u03BC")
    ax[key1%6][2].imshow(res2)
    ax[key1%6][2].set_title("$\u03BC + v_{1}\u221A\u03BB_{1}$")
plt.savefig("../results/pmv_digit6_9.jpg",dpi=600)
plt.close();

#### Q5

In [44]:
arrays={}
mat = h5py.File('../data/mnist.mat') #Process data
for k, v in mat.items():
    arrays[k] = np.array(v, dtype=np.float64)
for i in range(len(arrays['digits_train'])):
    arrays['digits_train'][i]=np.transpose(arrays['digits_train'][i])
for i in range(len(arrays['digits_test'])):
    arrays['digits_test'][i]=np.transpose(arrays['digits_test'][i])

In [45]:
def reduce_dimensions(digits_train, labels_train): 
    #Function to find 84 principal eigenvalues and corresponding eigenvectors
    digits=[[] for i in range(10)] #Store data of each digit
    digit_count=[0 for i in range(10)] #Number of traincases for each digit
    for i in range(len(digits_train)):
        digit_count[int(labels_train[i])]+=1
        digits[int(labels_train[i])].append(digits_train[i])

    meanmat=[np.zeros((28*28,1)) for i in range(len(digits))] #Mean vector for each digit

    for digit in range(len(digits)): #Compute mean
        for i in range(digit_count[digit]):
            meanmat[digit]+=np.reshape(digits[digit][i],(28**2,1))
        meanmat[digit]/=digit_count[digit]
        
    covmat=[np.zeros((28*28,28*28)) for i in range(len(digits))] #Covariance matrix for each digit
    for digit in range(len(digits)): #Compute covariance matrix
        for i in range(digit_count[digit]):
            covmat[digit]+=np.matmul(np.reshape(digits[digit][i],(28*28,1)),np.transpose(np.reshape(digits[digit][i],(28*28,1))))
        covmat[digit]/=digit_count[digit]
        covmat[digit]-=np.matmul(meanmat[digit],np.transpose(meanmat[digit]))
    
    evecs_all=[] #List of list of eigenvectors of each digit
    evalues_all=[] #List of list of eigenvalues of each digit
    for i in range(10):
        evalues, evecs = np.linalg.eigh(covmat[i]) #Find eigenvalues and eigenvectors for each digit
        evec_eval_pair = {evalues[j]:evecs[:,j] for j in range(len(evalues))}
        evalues = np.flip(np.sort(evalues)) #Sort eigenvalues
        evalues = evalues[0:84] #get top 84 eigenvalues
        evecs = np.array([evec_eval_pair[evalue] for evalue in evalues]) #get corresponding eigenvector
        evecs_all.append(evecs)
        evalues_all.append(evalues)

    return (evecs_all, evalues_all, meanmat) #Return tuple of eigenvectors & eigenvalues of all digits and the mean of all digits
    

In [46]:
(evecs, evals, meanmat) = reduce_dimensions(arrays['digits_train'], arrays['labels_train'][0])
#Find first 84 principal modes of variation for all digits from 0-9

In [47]:
def compute_coordinates(image, image_label, evecs, m): 
    return np.matmul(np.transpose(image-m),np.transpose(evecs[image_label]))
#Function to reduce dimensionality of each image and compute those 84 co-ordinates
#Takes input an image array, label of the image, and top 84 eigenvectors of that corresponding digit
#generated from reduce_dimensions function
#m denotes mean vector of that digit

In [48]:
digits_test = arrays['digits_test'] #Test digits raw dataset
labels_test = arrays['labels_test'][0] #Labels corresponding to those test digits
digits=[[] for i in range(10)] #Test digits dataset sorted according to digits
digit_count=[0 for i in range(10)] #Number of testcases for each digit
for i in range(len(digits_test)):
    digit_count[int(labels_test[i])]+=1 #Increment corresponding digit's count
    digits[int(labels_test[i])].append(digits_test[i]) #Add digit data to the corresponding index

#Multiple plots for cleaner presentation
fig,ax = plt.subplots(3,2,figsize=(10,15))
for digit in range(3): #For digits 0-2
    coords = compute_coordinates(np.reshape(digits[digit][0],(28*28,1)),digit,evecs, meanmat[digit]) #Reduce dimensionality to 84
    reconstructed = np.reshape(np.matmul(coords,evecs[digit])+np.transpose(meanmat[digit]),(28,28)) #Reconstruct image from those 84 dimensions
    ax[digit][0].imshow(digits[digit][0]) #Show original image
    ax[digit][1].imshow(reconstructed) #Show reconstructed image
    ax[digit][0].set_title(f"Digit {digit} Original image")
    ax[digit][1].set_title(f"Digit {digit} Reconstructed image")
plt.savefig("../results/reconstructimgs0_2.jpg",dpi=500)
plt.close()

fig,ax = plt.subplots(3,2,figsize=(10,15))
for digit in range(3,6): #For digits 3-5
    coords = compute_coordinates(np.reshape(digits[digit][0],(28*28,1)),digit,evecs, meanmat[digit])
    reconstructed = np.reshape(np.matmul(coords,evecs[digit])+np.transpose(meanmat[digit]),(28,28))
    ax[digit%3][0].imshow(digits[digit][0])
    ax[digit%3][1].imshow(reconstructed)
    ax[digit%3][0].set_title(f"Digit {digit} Original image")
    ax[digit%3][1].set_title(f"Digit {digit} Reconstructed image")
plt.savefig("../results/reconstructimgs3_5.jpg",dpi=500)
plt.close()

fig,ax = plt.subplots(4,2,figsize=(10,20))
for digit in range(6,10): #For digits 6-9
    coords = compute_coordinates(np.reshape(digits[digit][0],(28*28,1)),digit,evecs, meanmat[digit])
    reconstructed = np.reshape(np.matmul(coords,evecs[digit])+np.transpose(meanmat[digit]),(28,28))
    ax[digit%6][0].imshow(digits[digit][0])
    ax[digit%6][1].imshow(reconstructed)
    ax[digit%6][0].set_title(f"Digit {digit} Original image")
    ax[digit%6][1].set_title(f"Digit {digit} Reconstructed image")
plt.savefig("../results/reconstructimgs6_9.jpg",dpi=500)
plt.close();

#### Q6

In [49]:
from pathlib import Path
from PIL import Image
import scipy

In [50]:
meandat=np.zeros((19200,1)) #Mean vector of fruits
dat=[] #Dataset of all fruit images
files=Path("../data/data_fruit").glob('*')
for file in files:
    file_ext=Path(file).suffix
    if(file_ext!=".txt"):
        image=Image.open(file)
        az=np.asarray(image)
        dat.append(np.reshape(az, (19200,1)))
        meandat+=np.reshape(az, (19200,1))
meandat/=len(dat) #Computed mean

In [51]:
dat1=[] #Computing covariance
files=Path("../data/data_fruit").glob('*')
for file in files:
    file_ext=Path(file).suffix
    if(file_ext!=".txt"):
        image=Image.open(file) 
        az=np.asarray(image, dtype=np.float64) #open image and read into an array
        az=np.reshape(az,(19200,1)) #reshape the array
        az-=meandat #subtract the mean from it
        dat1.append(az)
dat1=np.reshape(dat1,(16,19200))
dat1=np.transpose(dat1)
dat1cov=np.matmul(dat1,np.transpose(dat1)) #Compute covariance matrix
dat1cov/=(len(dat)-1) #(n-1) for sample covariance estimator

In [52]:
eigenval,eigenvec=scipy.sparse.linalg.eigsh(dat1cov, k=4, which='LA') #compute top 4 principal eigenvectors & eigenvalues

In [53]:
fig,ax=plt.subplots(1,5, figsize=(15,3)) #Plot the mean & top 4 eigenvector images
ax[0].imshow(np.reshape(meandat,(80,80,3))/255)
ax[0].set_title("Mean Image")
for i in range(4):
    acz=np.reshape(eigenvec[:,3-i], (19200,1))
    acz=(acz-np.min(acz))/(np.max(acz)-np.min(acz))
    ax[i+1].imshow(np.reshape(acz,(80,80,3)))
ax[1].set_title("$1^{st}$ Principal Eigenvector")
ax[2].set_title("$2^{nd}$ Principal Eigenvector")
ax[3].set_title("$3^{rd}$ Principal Eigenvector")
ax[4].set_title("$4^{th}$ Principal Eigenvector")
plt.savefig("../results/4eigv_imgs.jpg",dpi=900)
plt.close();

In [54]:
eval10=scipy.sparse.linalg.eigsh(dat1cov, k=10, return_eigenvectors=False, which='LA')
#Compute top 10 eigenvalues

In [55]:
xs=[i for i in range(1,11)]

In [56]:
plt.figure(figsize=(8,5)) #Plot the top 10 eigenvalues in sorted order
plt.plot(xs,np.flip(eval10), 'ro')
plt.plot(xs,np.flip(eval10), 'b--', lw=0.5)
plt.xlabel("Index of eigenvalue")
plt.ylabel("Eigenvalue")
plt.title("Top 10 Eigenvalues")
plt.savefig("../results/top10eig.jpg",dpi=800)
plt.close();

In [57]:
meandat=meandat.astype(np.float64)
distances=[]
for i in range(4):
    eigenvec[:,i]=eigenvec[:,i]/np.linalg.norm(eigenvec[:,i]) #Normalize top 4 eigenvectors
fig,ax=plt.subplots(4,8,figsize=(20,12))
for j in range(16):
    img = np.reshape(dat[j], (19200,1)).astype(np.float64) #Read images
    img-=meandat #Subtract mean from them
    img_flattened = np.reshape(img,(19200,1)) #Flatten them into a 19200 dimensional column vector
    coords = np.matmul(np.transpose(img_flattened),eigenvec) #Compute coefficients of linear combination
    
    distances.append(np.linalg.norm(coords))
    
    #Coefficients are calculated by projecting the image vector onto the eigenvectors and computing the
    #directional components
    img_reconstructed = np.matmul(coords, np.transpose(eigenvec)) 
    #Reconstruct the image by adding the linear combination of top 4 eigenvectors to the mean image vector
    img_reconstructed = np.reshape(meandat+np.transpose(img_reconstructed),(80,80,3))
    img_reconstructed = (img_reconstructed-np.min(img_reconstructed))/(np.max(img_reconstructed)-np.min(img_reconstructed))
    ax[j//4][2*(j%4)].imshow(np.reshape(dat[j], (80,80,3))) #Display the original and reconstructed images
    ax[j//4][2*(j%4)].set_title("Original Image")
    ax[j//4][2*(j%4)+1].set_title("Reconstructed Image")
    ax[j//4][2*(j%4)+1].imshow(img_reconstructed)
plt.savefig("../results/reconstructfruits.jpg",dpi=400)
plt.close();

In [58]:
meandat = meandat.astype(np.float32)
seeds = [25,81,100] # set of seeds used to generate representative images
fig,ax=plt.subplots(1,3,figsize=(30,10))
for i in range(3):
    np.random.seed(seeds[i])
    newcoords = np.random.normal(size=(1,4),scale=np.sqrt(eigenval)) # random sample from multivariate gaussian distribution 
    newimg = np.matmul(newcoords, np.transpose(eigenvec)) # compute the image using the coordinates along principal eigenvectors
    newimg = np.reshape(meandat+np.transpose(newimg),(80,80,3))
    newimg = (newimg-np.min(newimg))/(np.max(newimg)-np.min(newimg)) #Rescale the sample image
    ax[i].imshow(newimg) # display new sampled image
    ax[i].set_title(f"Randomly Sampled Image {i+1}",size=20)
plt.savefig(f"../results/fruits_random_sample.jpg",dpi=400)
plt.close();